In [ ]:
from linebot import LineBotApi, WebhookHandler
from linebot.models import TextSendMessage, StickerSendMessage, ImageSendMessage, LocationSendMessage
import requests, json

import openai
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="##輸入openai key##",##輸入你的openai key
)

from firebase import firebase

token = '##輸入line token##'##輸入你的line token key
secret = '##輸入line secret##'##輸入你的line secret key



def linebot(request):
    body = request.get_data(as_text=True)
    json_data = json.loads(body)
    try:
        line_bot_api = LineBotApi(token)
        handler = WebhookHandler(secret)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        tk = json_data['events'][0]['replyToken']
        timestamp = json_data['events'][0]['timestamp']
        msg_type = json_data['events'][0]['message']['type']
        userID = json_data['events'][0]['source']['userId']
    
        if msg_type == 'text':
            msg = json_data['events'][0]['message']['text']
            url = 'https://xxxxxxxx.firebaseio.com/'##輸入你的firebase url
            fdb = firebase.FirebaseApplication(url, None)
            chatgpt = fdb.get('/chatgpt/',str(userID))

            if chatgpt == None:             
                messages = [{'content': '你是一個具有專業機車知識的客服人員', 'role': 'system'}]
                fdb.put_async('/chatgpt/',str(userID),messages)
            else:
                messages = [{'content': '你是一個具有專業機車知識的客服人員', 'role': 'system'}]
                # chatgpt = fdb.get('/chatgpt/',f'{userID}')
                messages+=chatgpt[-6:]

            if msg == '!reset':
                reply_msg = TextSendMessage(text='對話歷史紀錄已經清空！')
                line_bot_api.reply_message(tk,reply_msg)
                fdb.delete('/chatgpt/',str(userID))
            else:
                messages.append({"role":"user","content":msg})
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    max_tokens=128,
                    temperature=0.5,
                    messages=messages
                )
                ai_msg = response.choices[0].message.content.replace('\n','')
                messages.append({"role":"assistant","content":ai_msg})

                ####刪除system### 節省空間
                # content_to_remove = "你是一個具有專業機車知識的客服人員"
                #########################

                reply_msg = TextSendMessage(text=ai_msg)
                line_bot_api.reply_message(tk,reply_msg)
                fdb.put_async('/chatgpt/',str(userID),messages)
        else:
            reply_msg = TextSendMessage(text='你傳的不是文字訊息呦')
            line_bot_api.reply_message(tk,reply_msg)
    except Exception as e:
        detail = e.args[0]
        print(detail)
    return 'OK'
